<a href="https://colab.research.google.com/github/okura1406/radnlp-2024/blob/main/radnlp2024_gemini_2_0_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 説明

以下は放射線診断レポートからTNMステージを抽出するアンサンブル手法のサンプルコードです。

結果的には精度は0.8を上回らず精度の向上は見られなかった


# データの用意

まずは Colab 環境で扱えるようにデータを用意するところから始めます。データは Google Drive に用意して、これをマウントすることでアクセスできるようにします。

In [ ]:
# Google Drive をマウントするためのコード
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import re
import pandas as pd

# --------------------------------
# data directory の設定
# --------------------------------
data_dir = "/content/drive/MyDrive/"

# CSV を読み込み
valid_df = pd.read_csv(os.path.join(data_dir, "label.csv"))

# CSV の中身を確認
display(valid_df.head(5))

,id,t,n,m
0,147290,T2a,N0,M0
1,241752,Tis,N0,M0
2,876951,T2a,N0,M0
3,923073,T2a,N0,M0
4,1600422,T1c,N0,M0


# モデルの用意



In [ ]:
# GPU の確認.
!nvidia-smi

Thu Jan 23 14:09:01 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# -----------------------
# レポート本文を取得する関数
# -----------------------
def get_report(report_id):
    """
    '123' のような ID に対応する '123.txt' ファイルを data_dir から読み込む。
    """
    txt_path = os.path.join(data_dir, f"{report_id}.txt")

    # ファイルが存在しない場合のエラー確認（任意）
    if not os.path.exists(txt_path):
        raise FileNotFoundError(f"ファイルが見つかりません: {txt_path}")

    with open(txt_path, "r", encoding="utf-8") as f:
        report = f.read()
    return report


# -----------------------
# 以下、Google Generative AI 関連のセットアップ
# -----------------------
!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

# GEMINI_API_KEY を鍵アイコンで設定してあると仮定
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

# テスト用にモデル一覧を表示する場合
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print("使用可能モデル:", m.name)

使用可能モデル: models/gemini-1.0-pro-latest
使用可能モデル: models/gemini-1.0-pro
使用可能モデル: models/gemini-pro
使用可能モデル: models/gemini-1.0-pro-001
使用可能モデル: models/gemini-1.0-pro-vision-latest
使用可能モデル: models/gemini-pro-vision
使用可能モデル: models/gemini-1.5-pro-latest
使用可能モデル: models/gemini-1.5-pro-001
使用可能モデル: models/gemini-1.5-pro-002
使用可能モデル: models/gemini-1.5-pro
使用可能モデル: models/gemini-1.5-pro-exp-0801
使用可能モデル: models/gemini-1.5-pro-exp-0827
使用可能モデル: models/gemini-1.5-flash-latest
使用可能モデル: models/gemini-1.5-flash-001
使用可能モデル: models/gemini-1.5-flash-001-tuning
使用可能モデル: models/gemini-1.5-flash
使用可能モデル: models/gemini-1.5-flash-exp-0827
使用可能モデル: models/gemini-1.5-flash-002
使用可能モデル: models/gemini-1.5-flash-8b
使用可能モデル: models/gemini-1.5-flash-8b-001
使用可能モデル: models/gemini-1.5-flash-8b-latest
使用可能モデル: models/gemini-1.5-flash-8b-exp-0827
使用可能モデル: models/gemini-1.5-flash-8b-exp-0924
使用可能モデル: models/gemini-2.0-flash-exp
使用可能モデル: models/gemini-exp-1206
使用可能モデル: models/gemini-exp-1121
使用可能モデル: models/gemini-exp-1

# 推論

では、今回の RadNLP のデータで本番の推論に移ります。


In [ ]:
!pip install regex
import regex
import json
import re

# -----------------------
# TNM 抽出用の関数群
# -----------------------
def parse_tnm_response(text):
    """TNM を一括抽出するための正規表現パーサー"""
    t_match = re.search(r'(Tis|T1mi|T1a|T1b|T1c|T2a|T2b|T3|T4|T0|TX)', text)
    n_match = re.search(r'(N0|N1|N2|N3|NX)', text)
    m_match = re.search(r'(M0|M1a|M1b|M1c|M1|MX)', text)
    # -----------------

    reason_matches = re.findall(r'理由[:：]\s*(.+?)(?=\n|$)', text)
    reason = " ".join(reason_matches) if reason_matches else "理由記載なし"

    # マッチした文字列（例: "T2a", "N0", "M1b"）をそのまま返す
    return (
        t_match.group(0) if t_match else "TX",
        n_match.group(0) if n_match else "NX",
        m_match.group(0) if m_match else "MX",
        reason
    )


def parse_single_stage(text, target):
    """単一ステージ（NやMなど）のみを抽出する正規表現パーサー"""

    # targetが"M"のときだけサブステージ対応のパターンを使用
    if target == "M":
        # M0, M1, M1a, M1b, M1c, MX まで対応
        pattern = r'(M0|M1a|M1b|M1c|M1)'
    else:
        # Nやその他(デフォルト)は従来のパターン
        # (Nなら N0,N1,N2,N3,NX を拾える: target='N' → 'N([0-9X])')
        pattern = fr'{target}([0-9X])'

    match = re.search(pattern, text)
    reason_match = re.search(r'理由[:：]\s*(.+?)(?=\n|$)', text)
    reason = reason_match.group(1) if reason_match else "理由記載なし"

    if match:
        # group(0)で "M1a" など全体を取得
        return match.group(0), reason
    else:
        # 見つからなければ "Mx" や "Nx" などで返す
        return f"{target}X", reason


def parse_final_response_as_json(text):
    """最終レスポンスを JSON としてパースする"""
    try:
        # JSON 形式で記述された部分を抽出 (regex を使用)
        json_str = regex.search(r'\{(?:[^{}]|(?R))*\}', text).group(0)
        # JSON をパース
        return json.loads(json_str)
    except (json.JSONDecodeError, AttributeError, TypeError):
        # パースエラーまたは JSON 部分が見つからない場合は、デフォルト値を返す
        print(f"JSON パースエラーまたは JSON 部分が見つかりません: {text}")
        return {
            "T": {"stage": "", "details": ""},
            "N": {"stage": "", "details": ""},
            "M": {"stage": "", "details": ""}
        }

def extract_tnm_ensemble(report_text):
    """
    与えられたレポート本文から TNM 分類を抽出し、
    一括抽出/個別抽出をあわせたアンサンブルプロンプトで最終解釈を返す。
    """
    # お好みのモデルに置き換えてください
    model = genai.GenerativeModel("models/gemini-2.0-flash-thinking-exp-1219")

    # 1. TNMを一括抽出
    tnm_prompt = f"""
    以下の文書は私自身が保有するデータであり、著作権に関する問題はありません。
【instruction】に含まれる TNM分類_第8版_2017年 の定義と、下記の判定フローを参照して、
肺癌の読影レポートを TNM分類 に照らし合わせて判定し、指定の形式（JSON）で出力してください。

【instruction】
{{
  "TNM分類_第8版_2017年": {{
    "T_原発腫瘍": {{
      "TX": "原発腫瘍の存在が判定できない、または喀痰・気管支洗浄液細胞診でのみ陽性で画像・気管支鏡では観察不能",
      "T0": "原発腫瘍を認めない",
      "Tis": "上皮内癌(carcinoma in situ)：肺野型の場合、充実成分径0mmかつ病変全体径≦30mm",
      "T1": {{
        "定義": "腫瘍の充実成分径≦30mm、肺または臓側胸膜に覆われ、主気管支への浸潤なし",
        "T1mi": "微小浸潤性腺癌：部分充実型、充実成分径≦5mm、病変全体径≦30mm",
        "T1a": "充実成分径≦10mm、Tis・T1mi以外",
        "T1b": "充実成分径＞10mmかつ≦20mm",
        "T1c": "充実成分径＞20mmかつ≦30mm"
      }},
      "T2": {{
        "定義": "充実成分径＞30mmかつ≦50mm、または充実成分径≦30mmでも以下の条件を満たす場合:\n1) 主気管支浸潤(気管分岐部は除く)\n2) 臓側胸膜浸潤\n3) 肺門まで連続する無気肺または閉塞性肺炎",
        "T2a": "充実成分径＞30mmかつ≦40mm",
        "T2b": "充実成分径＞40mmかつ≦50mm"
      }},
      "T3": {{
        "定義": "充実成分径＞50mmかつ≦70mm、または充実成分径≦50mmでも以下のいずれか:\n1) 壁側胸膜、胸壁(上部肺溝腫瘍含む)、横隔神経、心膜への直接浸潤\n2) 同一葉内の不連続な副腫瘍結節"
      }},
      "T4": {{
        "定義": "充実成分径＞70mm、または以下いずれか:\n1) 横隔膜、縦隔、心臓、大血管、気管、反回神経、食道、椎体、気管分岐部への浸潤\n2) 同側の異なる肺葉内に不連続な副腫瘍結節"
      }}
    }},
    "N_所属リンパ節": {{
      "NX": "所属リンパ節評価不能",
      "N0": "所属リンパ節転移なし",
      "N1": "同側の気管支周囲、肺門、肺内リンパ節転移（原発腫瘍への直接浸潤含む）",
      "N2": "同側縦隔、気管分岐下リンパ節転移",
      "N3": "対側縦隔、対側肺門、前斜角筋・鎖骨上窩リンパ節転移"
    }},
    "M_遠隔転移": {{
      "M0": "遠隔転移なし",
      "M1": {{
        "定義": "遠隔転移あり",
        "M1a": "対側肺内副腫瘍結節、胸膜・心膜結節、悪性胸水・悪性心嚢水（同側・対側問わず）",
        "M1b": "肺以外の一臓器への単発遠隔転移",
        "M1c": "肺以外の一臓器または多臓器への多発遠隔転移"
      }}
    }}
  }},

肺癌のTNM分類について、以下のフォーマットに従って出力してください。

【答えの候補】

T（原発腫瘍）:Tis, T1mi, T1a, T1b, T1c, T2a, T2b, T3, T4
N（リンパ節転移）:NX, N0, N1, N2, N3
M（遠隔転移）: M0, M1a, M1b, M1c

    フォーマット: T[]N[]M[] (例: T1aN0M0)
    理由付きで説明してください。
    レポート:
    {report_text}
    """
    tnm_response = model.generate_content(tnm_prompt).text
    t, n, m, reason_all = parse_tnm_response(tnm_response)

    # 2. N と M を独立して抽出
    n_prompt = f"""
    以下のレポートからリンパ節転移（N分類）のみを抽出してください。
    回答は【答えの候補】から選択してください。

    "N_所属リンパ節": {{
      "NX": "所属リンパ節評価不能",
      "N0": "所属リンパ節転移なし",
      "N1": "同側の気管支周囲、肺門、肺内リンパ節転移（原発腫瘍への直接浸潤含む）",
      "N2": "同側縦隔、気管分岐下リンパ節転移",
      "N3": "対側縦隔、対側肺門、前斜角筋・鎖骨上窩リンパ節転移"
    }},
　　【N分類推定のポイント】
　　　転移の有無と部位：転移が認められない場合はN0です。転移がある場合は、その部位に基づいてN1、N2、N3に分類します。
　　　対側・同側の区別：転移が同側か対側かを確認し、適切なNステージに分類します。
　　　複数部位への転移：複数のリンパ節部位に転移がある場合、最も高いNステージに分類します（例：N1とN2の転移がある場合はN2）。

    【答えの候補】
　　　N（リンパ節転移）:N0, N1, N2, N3

    フォーマット: N[] 理由: [説明]
    レポート:
    {report_text}
    """
    n_response = model.generate_content(n_prompt).text
    n_independent, reason_n = parse_single_stage(n_response, 'N')

    m_prompt = f"""
    以下のレポートから遠隔転移（M分類）のみを抽出してください。回答は【答えの候補】から選択してください。
    "M_遠隔転移": {{
      "M0": "遠隔転移なし",
      "M1": {{
        "定義": "遠隔転移あり",
        "M1a": "対側肺内副腫瘍結節、胸膜・心膜結節、悪性胸水・悪性心嚢水（同側・対側問わず）",
        "M1b": "肺以外の一臓器への単発遠隔転移",
        "M1c": "肺以外の一臓器または多臓器への多発遠隔転移"
      }}
    }}
    【答えの候補】
　M（遠隔転移）: M0, M1a, M1b, M1c

    フォーマット: M[] 理由: [説明]
    レポート:
    {report_text}
    """
    m_response = model.generate_content(m_prompt).text
    m_independent, reason_m = parse_single_stage(m_response, 'M')

    # 3. アンサンブルによる最終解釈
    ensemble_prompt = f"""
放射線診断レポートと複数の抽出結果を統合して、以下の注意事項に沿って最終的な TNM 分類を決定してください

【T分類判定のステップ】
*腫瘍の有無の確認（最重要）
読影レポートを読み込み、最初に「腫瘍を認めない」、「悪性所見なし」などの明示的な記載の有無を厳格にチェック。
腫瘍を認めない場合 ⇒ T0。
原発腫瘍の存在が判定できない or 細胞診のみ陽性(画像で捉えられない) ⇒ TX。

*大まかなT分類の判定
レポートに “胸壁浸潤” “多発結節” など明らかなキーワードがあれば該当のT(T3/T4)に直行。
なければ 病変の充実成分径、GGO性状、主気管支浸潤の有無などを精査。
胸壁・大血管浸潤など Tを上げる決定的所見(T3/T4 所見)がないか優先チェックし、なければサイズ基準で T1 / T2 を振り分ける。
T1 or T2 と判定した場合、充実成分径の大きさから T1a, T1b, T1c, T2a, T2b に細かく振り分ける。
混同しやすい境界(30mm, 40mm, 50mm) は特に注意。
例: 30mm 付近 ⇒ T1c と T2a の両方を再評価し、他の情報と合わせて最終決定。

*浸潤部位の確認
臓側胸膜 vs 壁側胸膜の記載ゆれにも注意し、壁側胸膜への明確な浸潤があればT3(さらに大きさ次第でT4)とする。
“大血管”“心臓”“椎体”などのキーワード ⇒ T4 を強く疑う。

【Tステージ誤り防止のための重要事項】
1. 早期がん（Tis, T1mi, T0）の過大評価に注意！
   充実成分径が非常に小さい(≦5mm)場合は、安易に T1a〜T1c とせず T1mi や Tis の可能性を優先検討。
   Tis: 充実成分径0 mm、かつ病変全体30 mm以下。(充実成分径がわずかでも認められる場合はTisの可能性を排除)
   T1mi: 部分充実 (mixed GGO) で、solid 部分≦5 mm、全体≦30 mm。(充実成分径が5mmを超えている場合はT1a以降を検討)
   画像所見で浸潤が明らかでない場合 ⇒ まず早期がんの可能性を強く意識し、過大評価を避ける。

2. 進行がん（T3, T4）の過小評価に注意！　大径腫瘍・浸潤所見を見落とさない
   腫瘍が50mmを超える場合 ⇒ T3 以上を強く検討。
   70mm超や胸壁・大血管・心臓などへの浸潤が記載されていればT4の可能性大。
   壁側胸膜浸潤、胸壁侵犯、横隔膜・大血管・気管分岐部浸潤などのキーワードを見落とさず、T4を積極的に検討する。

【判定において特に注意すべき点】
"GGO関連":
  純粋GGO(pure GGO)か部分的solidを含むmixed GGOかを明確に抽出し、充実成分の径を厳密に評価。
  mixed GGOで充実成分≦5mmかつ病変全体≦30mm なら T1mi を最優先で検討。
  pure GGO (充実成分0mm)で病変全体30mm以下なら Tis をまず検討。

"副腫瘍結節":
  同一葉内ならT3、同側の別葉ならT4、対側ならM1a等、位置関係を正しくチェック。
  “不連続な結節”か単なる“直接浸潤”かの区別にも留意する。

"主要気管支・縦隔浸潤":
  気管分岐部まで及ぶか、胸壁・大血管・横隔膜・心臓などへの直接浸潤があれば、T4を優先的に考慮。

"既知病変と新規原発巣":
  レポートで「既知の増悪」「再発疑い」等があれば、T0判定(腫瘍を認めない) の可能性も忘れず検討。

【最終チェック(強制ルール)】
pure GGO が T1c 以上になっていないか 再確認
「腫瘍なし」と明記があれば T0 に矛盾していないか
胸壁・大血管浸潤などのキーワードを再スキャンして T4 を見落としていないか
副腫瘍結節 位置の再チェックで T3/T4/M1a を付け忘れていないか
充実成分径が0mmの純粋GGOがT1a以上になっていないか
充実成分径≦5mmかつ全体径≦30mmのmixed GGOがT1miとして考慮されているか

【M分類のポイント】
遠隔転移の有無をまず確認します。
転移がある場合は、転移部位と転移の数に基づいてM1a、M1b、M1cに分類します。
特に、肺以外の臓器への転移の有無とその数を重視してください。

【N分類推定のポイント】
　　　転移の有無と部位：転移が認められない場合はN0です。転移がある場合は、その部位に基づいてN1、N2、N3に分類します。
　　　対側・同側の区別：転移が同側か対側かを確認し、適切なNステージに分類します。
　　　複数部位への転移：複数のリンパ節部位に転移がある場合、最も高いNステージに分類します（例：N1とN2の転移がある場合はN2）。

肺癌のTNM分類について、以下のフォーマットに従って出力してください。
JSON形式で T（原発腫瘍）、N（リンパ節転移）、M（遠隔転移）に関する情報をそれぞれ記載してください。
各値は明確な記載がない場合は最も近いと思われるものとしてください。
空欄にならないように必ず【答えの候補】から1つの候補に絞ってください！

{{
  "T": {{
    "stage": "候補の中から選択",
    "details": "原発腫瘍のサイズや特徴の記載"
  }},
  "N": {{
    "stage": "候補の中から選択",
    "details": "リンパ節転移の有無や部位の記載"
  }},
  "M": {{
    "stage": "候補の中から選択",
    "details": "遠隔転移の有無や転移先の記載"
  }}
}}

【答えの候補】

T（原発腫瘍）:Tis, T1mi, T1a, T1b, T1c, T2a, T2b, T3, T4
N（リンパ節転移）:N0, N1, N2, N3
M（遠隔転移）: M0, M1a, M1b, M1c

レポート内容:
{report_text}
抽出結果:
- 一括抽出: {t}{n}{m} (理由: {reason_all})
- 独立抽出 N: {n_independent} (理由: {reason_n})
- 独立抽出 M: {m_independent} (理由: {reason_m})

矛盾がある場合は画像所見を踏まえて最も妥当な結果を選択し、
最終的な TNM 分類とその詳細な理由を説明してください。
"""
    final_response = model.generate_content(
        ensemble_prompt,
        generation_config=genai.types.GenerationConfig(
            temperature=0.3,
            max_output_tokens=2000
        )
    ).text

 # JSON パース結果を格納する辞書を初期化
    parsed_result = {
        "T": {"stage": "", "details": ""},
        "N": {"stage": "", "details": ""},
        "M": {"stage": "", "details": ""}
    }

    try:
        # JSON パースを試みる
        parsed_result = parse_final_response_as_json(final_response)
    except json.JSONDecodeError as e:
        # JSON パースに失敗した場合、エラーメッセージを出力
        print(f"JSON パースエラー: {e}, レスポンス: {final_response}")

    # パース結果またはデフォルト値を返す
    return {
        "initial_extraction": f"{t}{n}{m}",  # 一括抽出結果
        "n_individual": n_independent,         # 独立抽出 N
        "m_individual": m_independent,         # 独立抽出 M
        "final_tnm": parsed_result             # 最終的な TNM 分類
    }

# ----------------------------------------------------------------------
# ここから label.csv にある id を使って、TXT ファイルを読み込み → 推論
# ----------------------------------------------------------------------

for idx, row in valid_df.iterrows():
    current_id = row["id"]

    report_text = get_report(current_id)
    print(f"\n=== ID: {current_id} のレポートを解析します ===")
    print("レポート本文:")
    print(report_text)

    # TNM 抽出を実行
    result = extract_tnm_ensemble(report_text)

    # 抽出結果を表示
   # print("---- 抽出結果 ----")
    print("一括抽出での TNM       :", result["initial_extraction"])
    print("独立抽出 N (N分類のみ):", result["n_individual"])
    print("独立抽出 M (M分類のみ):", result["m_individual"])
    print("\n--- 最終TNM分類 (モデル回答) ---")
    print(result["final_tnm"])

    # 1件だけ処理したらループを抜ける
    break



=== ID: 147290 のレポートを解析します ===
レポート本文:
左肺門部に 37mm 大の腫瘤影を認め、ご指摘の肺癌が疑われます。
縦隔に有意なリンパ節腫大は認めません。
胸水はありません。
背部皮下に腫瘤を認め、粉瘤などと思われます。
一括抽出での TNM       : T2aN0M0
独立抽出 N (N分類のみ): N0
独立抽出 M (M分類のみ): M0

--- 最終TNM分類 (モデル回答) ---
{'T': {'stage': 'T2a', 'details': '左肺門部に37mm大の腫瘤影を認めます。充実成分径は不明ですが、腫瘤径からT2aに分類しました。'}, 'N': {'stage': 'N0', 'details': '縦隔に有意なリンパ節腫大は認めません。'}, 'M': {'stage': 'M0', 'details': '遠隔転移を疑わせる所見はありません。背部皮下の腫瘤は粉瘤の可能性が示唆されています。'}}


全データに対して推論を実行して、サブミッションのための csv を作っていきます。

In [ ]:
import pandas as pd
import time
import re
import google.generativeai as genai
from tqdm import tqdm
from google.colab import files


# 結果を格納するリスト
results = []

# tqdm を使うと進捗バーが出ます（不要なら外してOK）
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df), desc="Processing rows"):
    current_id = row["id"]
    print(f"\n=== ID: {current_id} のレポートを解析します ===")

    # レポート本文を取得
    report_text = get_report(current_id)
    print("レポート本文:")
    print(report_text)

    # 再トライの最大回数
    max_retries = 3
    attempt = 0
    last_error = None
    success = False

    # 3回まで再トライし、final_tnm (JSON) が正常にパースできるかを確認
    while attempt < max_retries:
        attempt += 1
        try:
            # (1) extract_tnm_ensemble を実行
            result = extract_tnm_ensemble(report_text)

            # (2) final_tnm の中身を取り出す (パース失敗/空の場合は後段で検知)
            tnm_dict = result.get("final_tnm", {})

            # (3) T, N, M の stage を取得
            T_stage = tnm_dict.get("T", {}).get("stage", "")
            N_stage = tnm_dict.get("N", {}).get("stage", "")
            M_stage = tnm_dict.get("M", {}).get("stage", "")

            # もし全て空だったらパース失敗とみなす or 何らかの異常とみなし再トライする想定も可
            # ここでは「少なくとも T, N, M のどれかに stage が入っていれば成功」とする例
            # 全て空ならエラー扱いにする場合:
            if (not T_stage) and (not N_stage) and (not M_stage):
                # 「パース自体は成功したが、中身が空なのでやり直し」という扱い
                raise ValueError("TNM が全て空だったため、再トライします。")

            # ---- ここまで来れば成功とみなす ----
            success = True
            print("---- 抽出結果 ----")
            print("T_stage:", T_stage)
            print("N_stage:", N_stage)
            print("M_stage:", M_stage)

            # リストに格納
            results.append({
                "id": current_id,
                "T_stage": T_stage,
                "N_stage": N_stage,
                "M_stage": M_stage,
                "error": None  # 正常
            })

            # リクエスト過多を避けるため、必要に応じて待機（例: 15秒）
            time.sleep(15)
            # 正常終了したので while ループを抜ける
            break

        except Exception as e:
            last_error = e
            print(f"[{current_id}] {attempt}回目の推論またはパースでエラー: {e}")
            # 次の再トライ前に少し待機（例: 15秒）
            time.sleep(15)

    # 3回トライしても成功フラグが立たなかった場合は空欄を格納する
    if not success:
        print(f"[{current_id}] 3回再トライしましたが失敗したため、T/N/M は空欄として出力します。")
        results.append({
            "id": current_id,
            "T_stage": "",
            "N_stage": "",
            "M_stage": "",
            "error": str(last_error) if last_error else "Unknown error"
        })


# -----------------------------
# 結果を DataFrame に変換して保存
# -----------------------------
results_df = pd.DataFrame(results)
display(results_df.head(5))

# CSVファイルとして保存
csv_filename = "submission1.csv"
results_df.to_csv(csv_filename, index=False)
print(f"Done. 全件の推論結果を {csv_filename} に出力しました。")

# ファイルをダウンロード
files.download(csv_filename)


Processing rows:   0%|          | 0/54 [00:00<?, ?it/s]


=== ID: 147290 のレポートを解析します ===
レポート本文:
左肺門部に 37mm 大の腫瘤影を認め、ご指摘の肺癌が疑われます。
縦隔に有意なリンパ節腫大は認めません。
胸水はありません。
背部皮下に腫瘤を認め、粉瘤などと思われます。
---- 抽出結果 ----
T_stage: T2a
N_stage: N0
M_stage: M0


Processing rows:   2%|▏         | 1/54 [00:44<38:59, 44.15s/it]


=== ID: 241752 のレポートを解析します ===
レポート本文:
右下葉に 14×15mm  の限局性すりガラス影があります。
粗大な充実部分は認めません。
内部を血管が通過しています。
既知の肺癌と考えます。
縦隔リンパ節腫大は認めません。
胸水貯留は指摘できません。
撮影範囲の腹部臓器に粗大な異常は認めません。
---- 抽出結果 ----
T_stage: Tis
N_stage: N0
M_stage: M0


Processing rows:   4%|▎         | 2/54 [01:29<38:48, 44.78s/it]

...


=== ID: 4724041 のレポートを解析します ===
レポート本文:
左肺門部、左上葉 S2 に主座を置く⻑径 37mm の不整形腫瘤を認め、内部に空洞形成
を認めます。原発性肺癌を疑います。
左肺動脈、左上葉気管支に接していますが狭小化はありません。副腫瘍結節を認めません。明らかな縦隔浸潤、肺門部、縦隔リンパ節の病的腫大を認めません。
胸水は認めません。
撮像範囲の上腹部臓器に明らかな異常は認めません。
JSON パースエラーまたは JSON 部分が見つかりません: 思考プロセス：

1. **T分類判定ステップに従う:**
   - **腫瘍の有無の確認:** レポートに「腫瘍を認めない」、「悪性所見なし」の記載はない。したがってT0ではない。
   - **大まかなT分類の判定:**
     - 「胸壁浸潤」「多発結節」などのキーワードはレポートにない。
     - 「⻑径 37mm の不整形腫瘤」という記載から、腫瘍径が37mmであることがわかる。
     - 「左肺動脈、左上葉気管支に接していますが狭小化はありません。」から、明らかな浸潤所見はないと判断できる。
     - 胸壁・大血管浸潤などのTを上げる決定的所見はない。
     - サイズ基準でT1/T2を検討する。37
[4724041] 1回目の推論またはパースでエラー: TNM が全て空だったため、再トライします。
[4724041] 2回目の推論またはパースでエラー: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.
---- 抽出結果 ----
T_stage: T2a
N_stage: N0
M_stage: M0


Processing rows:  26%|██▌       | 14/54 [16:05<1:08:33, 102.83s/it]


=== ID: 4734929 のレポートを解析します ===
レポート本文:
左舌区から下葉にまたがる 37mm の空洞性腫瘤あり、既知の肺癌と思われます。
縦隔リンパ節は一塊となっている可能性は否定できません。胸水認めません。
---- 抽出結果 ----
T_stage: T2a
N_stage: N2
M_stage: M0


Processing rows:  28%|██▊       | 15/54 [16:59<57:17, 88.15s/it]   


=== ID: 4924173 のレポートを解析します ===
レポート本文:
右肺下葉 S10 に長径 1.5cm のすりガラス状結節を認め、既知の肺腺癌を疑います。
内部に充実成分は認めず、Tis と考えます。
両肺に陳旧性炎症瘢痕を疑う索状影が散見されます。
左肺下葉胸膜下に多角形の小結節を認めます。炎症性結節を疑います。
他肺野に活動性炎症ないし腫瘍性病変は指摘できません。
縦隔、肺門、腋窩リンパ節腫大は認めません。
少量の右胸水、心嚢液貯留を認めます。
撮像範囲の腹部臓器に有意所見は指摘できません。
---- 抽出結果 ----
T_stage: Tis
N_stage: N0
M_stage: M0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')